In [1]:
import numpy as np
import math
import random

In [2]:
def create_city (cityfile):
    cityscape = {}
    with open(cityfile, "r") as city_file:
        for line in city_file:
            line = line.split()
            cityscape[line[0]] = (float(line[1]), float(line[2]))
    return cityscape

In [3]:
cities = create_city("cities.dat")

In [4]:
def find_distance (a, b):
    '''
    a,b are tuples (x-loc,y-loc)
    '''
    distance = math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    return distance

In [5]:
find_distance(cities["a"],cities["c"])

1.0

In [6]:
def calc_fitness (path, cityscape):
    '''
    path is a list of cities
    cityscape is the city dictionary
    '''
    fitness = 0.0
    for i in range(0, len(path)-1):
        fitness += find_distance(cityscape[path[i]], cityscape[path[i+1]])                      
    fitness += find_distance(cityscape[path[len(path)-1]], cityscape[path[0]])
                
    return fitness

In [7]:
path = []
for i in range(97,115):
    path.append(str(chr(i)))
fit_ = calc_fitness(path, cities)

In [8]:
def roll_two (l):
    i = 0
    j = 0
    while(i==j):
        i = random.randint(0,l-1)
        j = random.randint(0,l-1)
    return i,j

In [9]:
def random_config (city):
    '''
    takes the city dictionary to create initial configuration
    '''
    rand_config = list(city.keys())
    
    x = random.randint(1,20)
    
    for k in range(0, x):
        
        i,j = roll_two(len(rand_config))
            
        temp = rand_config[i]
        rand_config[i] = rand_config[j]
        rand_config[j] = temp
    
    return rand_config

In [10]:
def movement(config, i, j):
    
    temp = config[i]
    config[i] = config[j]
    config[j] = temp

    return config

In [48]:
def accept_reject(current_config, update_config, temperature, city):
    e1 = float(calc_fitness(current_config, city))
    e2 = float(calc_fitness(update_config, city))
    delta_e = e2-e1
    
    if delta_e < 0:
        return True
    elif random.random() < math.exp((-1*delta_e)/temperature):
        return True
    else:
        return False

In [79]:
def find_initial_temperature(city):
    
    init_config = random_config(city)
    config = init_config[:]
    
    delta_energies = []
    
    e1 = calc_fitness(config, city)
    
    for i in range(0,100):
        
        s1, s2 = roll_two(len(config))
        move_config = movement(config, s1, s2)
        e2 = calc_fitness(config, city)
        delta_energies.append(e2-e1)
    
    avg = sum(delta_energies)/float(100)
    return (-1*avg)/math.log(0.5)

In [87]:
print(find_initial_temperature(cities))

1.1115395593830324


In [60]:
QQQ = random_config(cities)
a = QQQ[:]
print(a)
a1 = calc_fitness(a, cities)
print(a1)

QQQ2 = movement(a, 4, 7)
b = QQQ2[:]
b2 = calc_fitness(b, cities)
print(b2)

print(b2-a1)
print(accept_reject(a, b, find_initial_temperature(cities), cities))
print(math.exp((-1*b2-a1)/float(16.31545285201128)))
print(random.random())

['a', 'b', 'd', 'o', 'l', 'f', 'g', 'q', 'i', 'n', 'k', 'h', 'e', 'j', 'c', 'p', 'm', 'r']
63.39972035180821
64.74445713623172
1.3447367844235032
False
0.00038813396032430947
0.16562295033037244


In [120]:
def simulated_annealing (data):
    
    #define the cityscape
    cities = create_city(data)
    
    #Initial Configuration
    config = random_config(cities)
    print(config, calc_fitness(config, cities))
    N = len(config)
    
    #Initial Temperature
    #temp = find_initial_temperature(cities)
    temp = 0.5
    
    #!!!
    accepted = 0
    tries = 0
    iteration = 0
    fitnesses = []
    freeze_condition = False
    
    while(not freeze_condition):

        #Elementary Configuration
        i, j = roll_two(N)
        transform = movement(config, i, j)

        #Accept/Reject
        if accept_reject(config, transform, temp, cities):
            config = transform[:]
            accepted += 1
            tries += 1
        else:
            #reject
            tries += 1
            
        if(accepted == 12*N or tries == 100*N):
            #equilibrium reached, track fitness and reduce temperature
            fitnesses.append(calc_fitness(config, cities))
            temp = temp*0.9
            iteration += 1
            accepted = 0
            tries = 0
            
            if iteration > 3: 
                if fitnesses[iteration-3] <= fitnesses[iteration-1]: #no improvement
                    if fitnesses[iteration-2] <= fitnesses[iteration-1]: #within the last
                        if fitnesses[iteration-1] <= fitnesses[iteration-1]: #three temperature steps
                            freeze_condition = True
    
    return config, calc_fitness(config, cities)

In [139]:
print(simulated_annealing("cities.dat"))

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'p', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'h', 'q', 'r'] 65.46568149349277
(['k', 'a', 'g', 'b', 'r', 'n', 'c', 'o', 'm', 'q', 'd', 'h', 'l', 'j', 'p', 'e', 'i', 'f'], 73.74221285695721)
